## Work
1. 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現

In [6]:
import os
import keras

# 本作業可以不需使用 GPU, 將 GPU 設定為 "無" (若想使用可自行開啟)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [7]:
train, test = keras.datasets.cifar10.load_data()

In [8]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [9]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [10]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [11]:
"""Code Here 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現
設定超參數
"""

LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7]

In [12]:
results_nesterov_true = {}
results_nesterov_false = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""

for momentum in MOMENTUM:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with MOMENTUM = %f" % (momentum))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=momentum)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "mom-%s" % str(momentum)
    results_nesterov_true[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

    
for momentum in MOMENTUM:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with MOMENTUM = %f" % (momentum))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=False, momentum=momentum)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "mom-%s" % str(momentum)
    results_nesterov_false[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

W0726 11:53:56.244796 4565833152 deprecation_wrapper.py:118] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0726 11:53:56.370087 4565833152 deprecation_wrapper.py:118] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Experiment with MOMENTUM = 0.950000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


W0726 11:53:56.487043 4565833152 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 8s 151us/step - loss: 2.0066 - acc: 0.2846 - val_loss: 1.8461 - val_acc: 0.3425
Epoch 2/50
50000/50000 [==============================] - 7s 132us/step - loss: 1.7953 - acc: 0.3694 - val_loss: 1.7448 - val_acc: 0.3881
Epoch 3/50
50000/50000 [==============================] - 7s 138us/step - loss: 1.7147 - acc: 0.4005 - val_loss: 1.6862 - val_acc: 0.4073
Epoch 4/50
50000/50000 [==============================] - 6s 126us/step - loss: 1.6544 - acc: 0.4187 - val_loss: 1.6321 - val_acc: 0.4206
Epoch 5/50
50000/50000 [==============================] - 7s 136us/step - loss: 1.6069 - acc: 0.4367 - val_loss: 1.6052 - val_acc: 0.4339
Epoch 6/50
50000/50000 [==============================] - 8s 153us/step - loss: 1.5685 - acc: 0.4496 - val_loss: 1.5659 - val_acc: 0.4474
Epoch 7/50
50000/50000 [==============================] - 7s 144us/step - loss: 1.5334 - acc: 0.4617 - val_loss: 1.5442 - 

Epoch 2/50
50000/50000 [==============================] - 7s 146us/step - loss: 1.8660 - acc: 0.3429 - val_loss: 1.8234 - val_acc: 0.3618
Epoch 3/50
50000/50000 [==============================] - 8s 151us/step - loss: 1.7904 - acc: 0.3725 - val_loss: 1.7661 - val_acc: 0.3864
Epoch 4/50
50000/50000 [==============================] - 7s 148us/step - loss: 1.7375 - acc: 0.3921 - val_loss: 1.7235 - val_acc: 0.4013
Epoch 5/50
50000/50000 [==============================] - 8s 154us/step - loss: 1.6987 - acc: 0.4059 - val_loss: 1.6799 - val_acc: 0.4141
Epoch 6/50
50000/50000 [==============================] - 8s 151us/step - loss: 1.6666 - acc: 0.4174 - val_loss: 1.6511 - val_acc: 0.4272
Epoch 7/50
50000/50000 [==============================] - 7s 144us/step - loss: 1.6379 - acc: 0.4286 - val_loss: 1.6359 - val_acc: 0.4286
Epoch 8/50
50000/50000 [==============================] - 7s 146us/step - loss: 1.6116 - acc: 0.4352 - val_loss: 1.6092 - val_acc: 0.4347
Epoch 9/50
50000/50000 [==========

...

50000/50000 [==============================] - 6s 125us/step - loss: 1.6267 - acc: 0.4302 - val_loss: 1.6248 - val_acc: 0.4312
Epoch 9/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6033 - acc: 0.4379 - val_loss: 1.5992 - val_acc: 0.4412
Epoch 10/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.5820 - acc: 0.4445 - val_loss: 1.5895 - val_acc: 0.4382
Epoch 11/50
50000/50000 [==============================] - 6s 126us/step - loss: 1.5650 - acc: 0.4513 - val_loss: 1.5767 - val_acc: 0.4495
Epoch 12/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.5434 - acc: 0.4590 - val_loss: 1.5670 - val_acc: 0.4459
Epoch 13/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.5271 - acc: 0.4649 - val_loss: 1.5425 - val_acc: 0.4564
Epoch 14/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.5132 - acc: 0.4685 - val_loss: 1.5287 - val_acc: 0.4632
Epoch 15/50
50000/50000 [===============

50000/50000 [==============================] - 6s 124us/step - loss: 1.6547 - acc: 0.4221 - val_loss: 1.6516 - val_acc: 0.4197
Epoch 10/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6377 - acc: 0.4267 - val_loss: 1.6397 - val_acc: 0.4207
Epoch 11/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6211 - acc: 0.4333 - val_loss: 1.6234 - val_acc: 0.4338
Epoch 12/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6077 - acc: 0.4373 - val_loss: 1.6195 - val_acc: 0.4273
Epoch 13/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.5913 - acc: 0.4419 - val_loss: 1.5998 - val_acc: 0.4423
Epoch 14/50
50000/50000 [==============================] - 6s 126us/step - loss: 1.5762 - acc: 0.4484 - val_loss: 1.5887 - val_acc: 0.4455
Epoch 15/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.5657 - acc: 0.4524 - val_loss: 1.5754 - val_acc: 0.4473
Epoch 16/50
50000/50000 [==============

50000/50000 [==============================] - 6s 124us/step - loss: 1.6978 - acc: 0.4080 - val_loss: 1.6935 - val_acc: 0.4122
Epoch 11/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.6810 - acc: 0.4131 - val_loss: 1.6771 - val_acc: 0.4120
Epoch 12/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.6645 - acc: 0.4185 - val_loss: 1.6641 - val_acc: 0.4146
Epoch 13/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6498 - acc: 0.4240 - val_loss: 1.6463 - val_acc: 0.4232
Epoch 14/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6362 - acc: 0.4292 - val_loss: 1.6369 - val_acc: 0.4219
Epoch 15/50
50000/50000 [==============================] - 6s 124us/step - loss: 1.6229 - acc: 0.4342 - val_loss: 1.6243 - val_acc: 0.4279
Epoch 16/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.6103 - acc: 0.4370 - val_loss: 1.6188 - val_acc: 0.4298
Epoch 17/50
50000/50000 [==============

50000/50000 [==============================] - 6s 125us/step - loss: 1.7240 - acc: 0.3986 - val_loss: 1.7180 - val_acc: 0.3967
Epoch 12/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.7096 - acc: 0.4044 - val_loss: 1.7065 - val_acc: 0.4036
Epoch 13/50
50000/50000 [==============================] - 6s 127us/step - loss: 1.6949 - acc: 0.4097 - val_loss: 1.6891 - val_acc: 0.4086
Epoch 14/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.6811 - acc: 0.4149 - val_loss: 1.6788 - val_acc: 0.4145
Epoch 15/50
50000/50000 [==============================] - 6s 126us/step - loss: 1.6683 - acc: 0.4184 - val_loss: 1.6647 - val_acc: 0.4208
Epoch 16/50
50000/50000 [==============================] - 6s 127us/step - loss: 1.6573 - acc: 0.4229 - val_loss: 1.6588 - val_acc: 0.4210
Epoch 17/50
50000/50000 [==============================] - 6s 126us/step - loss: 1.6450 - acc: 0.4266 - val_loss: 1.6459 - val_acc: 0.4308
Epoch 18/50
50000/50000 [==============

50000/50000 [==============================] - 9s 175us/step - loss: 1.7260 - acc: 0.3964 - val_loss: 1.7225 - val_acc: 0.3999
Epoch 13/50
50000/50000 [==============================] - 8s 170us/step - loss: 1.7143 - acc: 0.4013 - val_loss: 1.7115 - val_acc: 0.4002
Epoch 14/50
50000/50000 [==============================] - 9s 188us/step - loss: 1.7012 - acc: 0.4067 - val_loss: 1.7063 - val_acc: 0.3998
Epoch 15/50
50000/50000 [==============================] - 8s 154us/step - loss: 1.6895 - acc: 0.4095 - val_loss: 1.6881 - val_acc: 0.4110
Epoch 16/50
50000/50000 [==============================] - 7s 141us/step - loss: 1.6780 - acc: 0.4137 - val_loss: 1.6782 - val_acc: 0.4141
Epoch 17/50
50000/50000 [==============================] - 8s 153us/step - loss: 1.6671 - acc: 0.4192 - val_loss: 1.6691 - val_acc: 0.4179
Epoch 18/50
50000/50000 [==============================] - 8s 150us/step - loss: 1.6570 - acc: 0.4218 - val_loss: 1.6576 - val_acc: 0.4208
Epoch 19/50
50000/50000 [==============

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_nesterov_true.keys()):
    plt.plot(range(len(results_nesterov_true[cond]['train-loss'])),results_nesterov_true[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_nesterov_true[cond]['valid-loss'])),results_nesterov_true[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_nesterov_true.keys()):
    plt.plot(range(len(results_nesterov_true[cond]['train-acc'])),results_nesterov_true[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_nesterov_true[cond]['valid-acc'])),results_nesterov_true[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results_nesterov_false.keys()):
    plt.plot(range(len(results_nesterov_false[cond]['train-loss'])),results_nesterov_false[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_nesterov_false[cond]['valid-loss'])),results_nesterov_false[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results_nesterov_false[cond]['train-acc'])),results_nesterov_false[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results_nesterov_false[cond]['valid-acc'])),results_nesterov_false[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()